In [273]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sklearn
from sklearn import metrics, tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier, plot_tree

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [274]:
xls = pd.ExcelFile('Dataset - LBP RA.xlsx')
dataframe = pd.read_excel(xls, 'Training Dataset')

In [275]:
# Mapping integer column
value_columns = ["Decreased_mobility"]
dataframe[value_columns] = dataframe[value_columns].astype("Int64")

# Mapping categories and boolean columns
categorical_columns = ["Treatment", "Weightloss_per_year"]

boolean_columns = [
    "Fever",
    "Sick_leave",
    "Earlier_hospitalization",
    "Workoverload",
    "Familiy_history",
    "Depression",
    "Stress",
    "Uses_analgesics",
    "Uses_corticosteroids",
    "Serious_disease",
    "Neurogenic_signals",
    "Continuous_pain",
    "Nocturnal_pain",
    "Loss_muscle_strength",
    "Trauma",
    "Failure_symptoms",
    "Incoordination",
    "Paidwork",
]

dataframe[categorical_columns] = dataframe[categorical_columns].astype("category")
dataframe[boolean_columns] = dataframe[boolean_columns].astype("boolean")

# Mapping ordinal columns 
age_mapping = {
    "0-19": 0,
    "20-29": 1,
    "30-39": 2,
    "40-49": 3,
    "50-59": 4,
    "60-69": 5,
    "70-79":6,
    ">=80": 7,
}

dataframe["Age"] = dataframe["Age"].replace(age_mapping)

ordinal_columns = [
    "Duration_of_pain",
    "Extremely_nervous",
    "Relationship_with_colleagues",
    "Irrational_thoughts_risk_lasting",
    "Irrational_thoughts_work",
    "Coping_strategy",
    "Kinesiophobia_physical_exercise",
    "Kinesiophobia_pain_stop",
    "Age",
    "neck_pain_intensity",
    "low_back_pain_intensity",
    "arm_left_pain_intensity",
    "arm_right_pain_intensity",
    "leg_left_pain_intensity",
    "leg_right_pain_intensity",
    "working_ability",
]

for column in ordinal_columns:
    dataframe[[column]] = dataframe[[column]].astype("Int64")
    dataframe[column].fillna(-1, inplace=True)
    dataframe[column] = pd.Categorical(dataframe[column], categories=sorted(dataframe[column].unique()), ordered=True)

print(dataframe.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1546 entries, 0 to 1545
Data columns (total 37 columns):
 #   Column                            Non-Null Count  Dtype   
---  ------                            --------------  -----   
 0   Treatment                         1546 non-null   category
 1   Fever                             1512 non-null   boolean 
 2   Duration_of_pain                  1546 non-null   category
 3   Sick_leave                        1546 non-null   boolean 
 4   Earlier_hospitalization           1546 non-null   boolean 
 5   Workoverload                      459 non-null    boolean 
 6   Familiy_history                   1546 non-null   boolean 
 7   Depression                        1546 non-null   boolean 
 8   Extremely_nervous                 1546 non-null   category
 9   Stress                            1546 non-null   boolean 
 10  Relationship_with_colleagues      1546 non-null   category
 11  Irrational_thoughts_risk_lasting  1546 non-null   catego

In [276]:
categorical_columns.remove("Treatment")
X_encoded = pd.get_dummies(dataframe[categorical_columns], drop_first=True)

X = pd.concat(
    [dataframe[value_columns + ordinal_columns + boolean_columns], X_encoded], axis=1
)

y = dataframe["Treatment"]

# Decision Tree Model
param_grid = {
    "max_depth": [5, 10, 15],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}
grid_search = GridSearchCV(
    DecisionTreeClassifier(), param_grid, cv=5, scoring="accuracy"
)
grid_search.fit(X, y)
best_params = grid_search.best_params_
print("Best Score:", grid_search.best_score_)

best_max_depth = grid_search.best_params_["max_depth"]
best_min_samples_split = grid_search.best_params_["min_samples_split"]
best_min_samples_leaf = grid_search.best_params_["min_samples_leaf"]

tree_model = DecisionTreeClassifier(
    max_depth=best_max_depth,
    min_samples_split=best_min_samples_split,
    min_samples_leaf=best_min_samples_leaf,
)

tree_predicted = cross_val_predict(tree_model, X, y, cv=5)
tree_model.fit(X, y)

# Evaluation

print("Simple k=10 K fold CV")

print("Decision Tree Model:")

print(classification_report(y, tree_predicted))

Best Score: 0.4379308904896127
Simple k=10 K fold CV
Decision Tree Model:
              precision    recall  f1-score   support

           1       0.45      0.48      0.47       659
           2       0.15      0.08      0.10       160
           3       0.02      0.02      0.02        65
           4       0.00      0.00      0.00        13
           5       0.44      0.48      0.46       649

    accuracy                           0.41      1546
   macro avg       0.21      0.21      0.21      1546
weighted avg       0.39      0.41      0.40      1546



In [277]:
class_labels = [str(label) for label in tree_model.classes_]
plt.figure(figsize=(135, 90))
plot_tree(tree_model,max_depth=5, feature_names=X.columns, class_names=class_labels, filled=True, rounded=True)
plt.show()

# Note that colors are based on the majority class in a leaf (with intensity being an indicator for how large this majority is over the others).

In [278]:
X = dataframe[[col for col in value_columns + ordinal_columns + categorical_columns + boolean_columns if col != "Treatment"]]
y = dataframe['Treatment'] 

# Decision Tree Model
tree_model = HistGradientBoostingClassifier(max_depth=10)
tree_predicted = cross_val_predict(tree_model, X, y, cv=5)

# Evaluation
print("Simple k=10 K fold CV")
print("Decision Tree Model:")
print(classification_report(y, tree_predicted))

Simple k=10 K fold CV
Decision Tree Model:
              precision    recall  f1-score   support

           1       0.45      0.44      0.44       659
           2       0.30      0.08      0.13       160
           3       0.22      0.03      0.05        65
           4       0.00      0.00      0.00        13
           5       0.44      0.59      0.51       649

    accuracy                           0.44      1546
   macro avg       0.28      0.23      0.23      1546
weighted avg       0.42      0.44      0.42      1546



c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
